In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#Visualizations library
import seaborn as sns 
import matplotlib.pyplot as plt


#Stats library
from scipy import stats
from scipy.stats import norm, skew 

#Preprocessing Libraries
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from scipy.special import boxcox1p
from sklearn.feature_extraction.text import CountVectorizer


# Modellling Librries
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LassoCV, Ridge, LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV 

import os
# print(os.listdir("../House Price Predictor"))

# Any results you write to the current directory are saved as output.

In [4]:
sales = pd.read_csv('kc_house_data.csv')

In [5]:
trainData = pd.read_csv('kc_house_train_data.csv')
testData = pd.read_csv('kc_house_test_data.csv')

In [7]:
for df in  [trainData, testData]:
    df['bedrooms_squared'] = df['bedrooms'] * df['bedrooms']
    df['bed_bath_rooms'] = df['bedrooms'] * df['bathrooms']
    df['log_sqft_living'] = np.log(df['sqft_living'])
    df['lat_plus_long'] = df['lat'] + df['long']

In [8]:
trainData

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340,5650,9,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690,7639,9,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,98028,47.7379,-122.233,2720,8062,4,2.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360,5000,16,12.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,98074,47.6168,-122.045,1800,7503,9,6.00,7.426549,-74.4282
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,0,98053,47.6561,-122.005,4760,101930,16,18.00,8.597851,-74.3489
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,0,98003,47.3097,-122.327,2238,6819,9,6.75,7.447168,-75.0173
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,0,98198,47.4095,-122.315,1650,9711,9,4.50,6.966024,-74.9055
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,0,98146,47.5123,-122.337,1780,8113,9,3.00,7.484369,-74.8247
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,0,98038,47.3684,-122.031,2390,7570,9,7.50,7.544332,-74.6626


In [13]:
meanVal = []
for feat in ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']:
    meanVal.append(testData[feat].mean())
    
print(meanVal)

[12.4466777015843, 7.5039016315913925, 7.550274679645921, -74.65333355403185]


In [15]:
model1 = LinearRegression()
model2 = LinearRegression()
model3 = LinearRegression()

In [22]:
feats1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
feats2 = feats1 + ['bed_bath_rooms']
feats3 = feats2 + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [18]:
model1.fit(trainData[feats1], trainData['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [23]:
model2.fit(trainData[feats2], trainData['price'])
model3.fit(trainData[feats3], trainData['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
model1.coef_

array([ 3.12258646e+02, -5.95865332e+04,  1.57067421e+04,  6.58619264e+05,
       -3.09374351e+05])

In [24]:
model2.coef_

array([ 3.06610053e+02, -1.13446368e+05, -7.14613083e+04,  6.54844630e+05,
       -2.94298969e+05,  2.55796520e+04])

In [25]:
model3.coef_

array([ 5.29422820e+02,  3.45142296e+04,  6.70607813e+04,  5.34085611e+05,
       -4.06750711e+05, -8.57050439e+03, -6.78858667e+03, -5.61831484e+05,
        1.27334900e+05])

In [29]:
output = testData.price

In [30]:
testData.drop('price', axis=1, inplace=True)

In [27]:
def getRSS(output, predictions):
    RSS = np.sum(np.square(output-predictions))
    return RSS

In [33]:
predict1 = model1.predict(testData[feats1])
predict2 = model2.predict(testData[feats2])
predict3 = model3.predict(testData[feats3])

In [50]:
predictTr1 = model1.predict(trainData[feats1])
predictTr2 = model2.predict(trainData[feats2])
predictTr3 = model3.predict(trainData[feats3])

In [47]:
RSSVal = []
for pred in [predict1, predict2, predict3]:
    RSSVal.append(getRSS(output, pred))

In [48]:
RSSVal

[225500469795490.34, 223377462976467.5, 259236319207180.12]

In [52]:
outputTr = trainData.price

In [53]:
RSSValTr = []
for pred in [predictTr1, predictTr2, predictTr3]:
    RSSValTr.append(getRSS(outputTr, pred))

In [54]:
RSSValTr

[967879963049545.8, 958419635074070.0, 903436455050477.8]